<a href="https://colab.research.google.com/github/MayankKhoria2007/codeX-2.0ai-ml/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# CNN Classification + Feature Map + Prediction Visualization
# ============================================================

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# ------------------------------------------------------------
#  Data Preprocessing
# ------------------------------------------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset_choice = "fashion"  # or "fashion"

if dataset_choice == "mnist":
    train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
    test_data = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
    class_names = [str(i) for i in range(10)]
else:
    train_data = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
    test_data = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)
    class_names = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# ------------------------------------------------------------
#  CNN Model
# ------------------------------------------------------------
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        f1 = x.clone()                    # feature map 1
        x = self.pool(x)

        x = self.relu(self.conv2(x))
        f2 = x.clone()                    # feature map 2
        x = self.pool(x)


        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x, f1, f2

# ------------------------------------------------------------
# 3️⃣ Training
# ------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 3
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs, _, _= model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {total_loss/len(train_loader):.4f}")

# ------------------------------------------------------------
#  Evaluate Accuracy
# ------------------------------------------------------------
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []
all_images = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs, _, _= model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds.append(predicted.cpu())
        all_labels.append(labels.cpu())
        all_images.append(images.cpu())

accuracy = 100 * correct / total
print(f"\n✅ Test Accuracy: {accuracy:.2f}%")

# Combine all batches
all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)
all_images = torch.cat(all_images)

# ------------------------------------------------------------
#  Visualize Feature Maps (first image)
# ------------------------------------------------------------
dataiter = iter(test_loader)
images, labels = next(dataiter)
image = images[0].unsqueeze(0).to(device)

with torch.no_grad():
    _, f1, f2 = model(image)

f1 = f1.cpu().squeeze().numpy()
f2 = f2.cpu().squeeze().numpy()

plt.figure(figsize=(2,2))
plt.title("Original Image")
plt.imshow(images[0][0], cmap="gray")
plt.axis("off")
plt.show()

plt.figure(figsize=(12, 4))
plt.suptitle("Feature Maps - Conv1")
for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(f1[i], cmap="gray")
    plt.axis("off")
plt.show()

plt.figure(figsize=(12, 4))
plt.suptitle("Feature Maps - Conv2")
for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(f2[i], cmap="gray")
    plt.axis("off")
plt.show()



# ------------------------------------------------------------
#  Show Sample Predictions (Correct + Misclassified)
# ------------------------------------------------------------
preds = all_preds.numpy()
labels = all_labels.numpy()
images = all_images.numpy()

correct_idx = np.where(preds == labels)[0]
wrong_idx = np.where(preds != labels)[0]

# Show 5 correct predictions
plt.figure(figsize=(10, 2))
plt.suptitle("✅ Correct Predictions")
for i, idx in enumerate(correct_idx[:5]):
    plt.subplot(1, 5, i+1)
    plt.imshow(images[idx][0], cmap="gray")
    plt.title(f"P:{class_names[preds[idx]]}\nT:{class_names[labels[idx]]}")
    plt.axis("off")
plt.show()

# Show 5 misclassified images
plt.figure(figsize=(10, 2))
plt.suptitle("❌ Misclassified Images")
for i, idx in enumerate(wrong_idx[:5]):
    plt.subplot(1, 5, i+1)
    plt.imshow(images[idx][0], cmap="gray")
    plt.title(f"P:{class_names[preds[idx]]}\nT:{class_names[labels[idx]]}")
    plt.axis("off")
plt.show()
